In [1]:
!pip install feedparser

     |████████████████████████████████| 81kB 3.2MB/s 
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-cp36-none-any.whl size=6067 sha256=e1d4484c4067da47e26880824934c1d687d6f49493f9451ac6fbf16977401405
  Stored in directory: /root/.cache/pip/wheels/f1/80/5a/444ba08a550cdd241bd9baf8bae44be750efe370adb944506a
Successfully built sgmllib3k


In [2]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 92kB 9.6MB/s 
     |████████████████████████████████| 460kB 43.4MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [3]:
import time
import re
import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import feedparser
from bs4 import BeautifulSoup
import urllib
import urllib.request as req
import requests
from konlpy.tag import Kkma, Okt, Komoran

okt = Okt()
from konlpy.utils import pprint
import warnings

warnings.filterwarnings("ignore")

In [58]:


href_list = []  # 기사 주소가 들어갈 리스트
TitDesc_list = []  # 제목 + 요약 내용 리스트
title_list = []  # 제목 리스트


def Crawling(keyword, start, end, pageNum):
    keyword = '+'.join(keyword.split(' '))

    last = False
    page_num = 1

    ds = start
    de = end
    while page_num != pageNum:
        url = "https://search.naver.com/search.naver?&where=news&query={0}&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=3&ds={1}&de={2}&docid=&nso=so:r,p:,a:all&mynews=1&cluster_rank=238&start={3}&refresh_start=0".format(
            keyword, ds, de, str(page_num))
        raw = requests.get(url)
        # print("raw : ",raw)
        html = raw.content
        # print(html)
        soup = BeautifulSoup(html, 'html.parser')
        ul = soup.find('ul', {'class':'list_news'})

        # ul = soup.find('ul', {'class': 'type01'})
        li_list = ul.findAll('li')
        for li in li_list:
            try:
              # soup.select_one('p.youngone#junu')
                new = li.select_one('div.news_area')
                # print(new)

                new_title = new.select_one('a.news_tit')
                naver_news_link = new.select_one('div.news_info')
                # print(naver_news_link)
                
# 예시) soup.select('p')[1] 를 입력 후 실행하면, 여러 개의 p태그 중에서 두 번째 p태그의 내용만 나옵니다.
                naver_news_url = li.select('a')[2]
                # print("li.select('a')[2] : ",li.select('a')[2])


                title = new_title['title']
                link = naver_news_url['href']
                # print(link)
                if(keyword in title and "네이버뉴스" in naver_news_url ):
                  link = link.replace('amp;', '')
                  # print(link)
                  href_list.append(link)
                  


            except AttributeError:
                pass

        # 마지막 페이지 주소 확인 (다음페이지 버튼이 없으면 종료페이지로 간주)
        page = soup.find('div', {'class': 'sc_page'})
        page_a_list = page.findAll('a')
        if '다음' in str(page_a_list[-1]):
            page_num += 1
        else:
            last = True


Crawling('동화약품', '2020.01.01', '2020.10.31', 50)

In [59]:
# print(href_list)
# duplicated url eliminate
href_list = set(href_list)
print("after set : ",href_list)

after set :  {'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=215&aid=0000865765', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=103&oid=011&aid=0003811437', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=008&aid=0004464466', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=103&oid=020&aid=0003296638', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=004&oid=346&aid=0000031590', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=018&aid=0004684467', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=004&oid=346&aid=0000032596', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=001&aid=0011744089', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=215&aid=0000894684', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=103&oid=011&aid=0003785248', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=004&oid=346&aid=0000033824', 'https:

In [60]:
def write_news_article_urls(output_file, urls):
    """
    기사 URL들을 출력 파일에 기록한다.
    :param output_file:
    :param urls:
    :return:
    """
    for url in urls:
        print(url, file=output_file)


In [61]:
def ext_news_article_urls(html):
    """
    주어진 html에서 기사 url을 추출하여 돌려준다.
    :param html:
    :return:
    """

    url_frags = re.findall('<a href="(.*?)"', html)
    news_article_urls=[]

    for url_frag in url_frags:
        if  "sid1=101&sid2=258" in url_frag and "aid" in url_frag:
            news_article_urls.append(url_frag)
        else :
            continue

    return news_article_urls


In [62]:
def write_html(output_file, html):
    """
    주어진 HTML 텍스트를 출력 파일에 쓴다.
    :param output_file:
    :param html:
    :return:
    """

    output_file.write("{}\n".format(html))
    output_file.write("@@@@@ ARTICLE DELMITER @@@@\n")

In [63]:

target_word = "동화약품" #input("Enter keyword : ")
target_date = "20200101"
output_file_name =  "/content/drive/MyDrive/Colab Notebooks/stock_ML/html"+target_word+"__"+target_date+".txt"#input("Enter output file name : ")

output_file = open(output_file_name, "w", encoding='utf-8')


In [64]:

for line in href_list:

    article_id = line[(len(line) - 24):len(line)]
    print_url = "http://news.naver.com/main/tool/print.nhn?" + article_id

    # article_id = line[(len(line) - 24):len(line)]
    # print_url = line


    user_agent = "'Mozilla/5.0"
    headers = {"User-Agent": user_agent}
    try:
      # print("Success!")
      response = requests.get(print_url, headers=headers)
      html = response.text
      write_html(output_file, html)

    except requests.exceptions.RequestException as error:
      print("Error: ", error)

output_file.close()



In [65]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
네이버 뉴스 기사 HTML에서 순수 텍스트 기사를 추출한다.
"""

import bs4
import time
import re
import requests
import os


ARTICLE_DELIMITER = "@@@@@ ARTICLE DELMITER @@@@\n"
TITLE_START_PAT = '<h3 class="font1">'
TITLE_END_PAT = '</h3>'
DATE_TIME_START_PAT = u'기사입력 <span class="t11">'
BODY_START_PAT = '<div class="article_body">'
BODY_END_PAT = '<div class="article_footer">'
TIDYUP_START_PAT = '<div class="article_footer">'


def read_html_article(html_file):
    """
    HTML 파일에서 기사 하나를 읽어서 돌려준다.
    :param html_file:
    :return:
    """

    lines = []
    for line in html_file:
        if line.startswith(ARTICLE_DELIMITER):
            html_text = "".join(lines).strip()
            return html_text
        lines.append(line)

    return None

def ext_title(html_text):
    """
    HTML 기사에서 제목을 추출하여 돌려준다.
    :param html_text:
    :return:
    """
    p = html_text.find(TITLE_START_PAT)
    q = html_text.find(TITLE_END_PAT)
    title = html_text[p + len(TITLE_START_PAT):q]
    title = title.strip()

    return title


def ext_date_time(html_text):
    """
    HTML 기사에서 날짜와 시간을 추출하여 돌려준다.
    :param html_text:
    :return:
    """
    start_p = html_text.find(DATE_TIME_START_PAT)+len(DATE_TIME_START_PAT)
    end_p = start_p + 10
    date_time = html_text[start_p:end_p]
    date_time = date_time.strip()

    return date_time

def strip_html(html_body):
    """
    HTML 본문에서 HTML 태그를 제거하고 돌려준다.
    :param html_body:
    :return:
    """
    page = bs4.BeautifulSoup(html_body, "html.parser")
    body = page.text

    return body

def tidyup(body):
    """
    본문에서 필요없는 부분을 자르고 돌려준다.
    :param body:
    :return:
    """

    p = body.find(TIDYUP_START_PAT)
    # print(p)
    body = body[:p]
    body = body.strip()

    return body

def ext_body(html_text):
    """
    HTML 기사에서 본문을 추출하여 돌려준다.
    :param html_text:
    :return:
    """

    p = html_text.find(BODY_START_PAT)
    q = html_text.find(BODY_END_PAT)
    html_body = html_text[p + len(BODY_START_PAT):q]
    len(html_body)
    href_inx = html_body.find('<a')
    # ^ M_$
    html_body = html_body[0:href_inx-1]
    html_body = html_body.replace("<br />","\n")
    html_body = html_body.strip()
    body = strip_html(html_body)
    body = tidyup(body)

    return body

def write_article(text_file, title, date_time, body):
    """
    텍스트 파일에 항목이 구분된 기사를 출력한다.
    :param text_file:
    :param title:
    :param date_time:
    :param body:
    :return:
    """

    text_file.write("{}\n".format(title))
    # text_file.write("{}\n".format(date_time))
    text_file.write("{}\n".format(body))
    # text_file.write("{}\n".format(ARTICLE_DELIMITER))

In [66]:
target_word = "동화약품" #input("Enter keyword : ")
target_date = "20200101"
html_file_name =  "/content/drive/MyDrive/Colab Notebooks/stock_ML/html"+target_word+"__"+target_date+".txt"
text_file_name ="/content/drive/MyDrive/Colab Notebooks/stock_ML/result"+target_word+"__"+target_date+".txt"

html_file = open(html_file_name, "r", encoding="utf-8")
text_file = open(text_file_name, "w", encoding="utf-8")

while True:

    html_text = read_html_article(html_file)

    if not html_text:
        break

    title = ext_title(html_text)
    date_time = ext_date_time(html_text)
    body = ext_body(html_text)
    write_article(text_file, title, date_time, body)

html_file.close()
text_file.close()
